In [3]:
import pandas as pd
import numpy as np
import numba as nb
import warnings
warnings.filterwarnings('ignore')

from tqdm.notebook import tqdm

This dataset is about purchases in a online shop, user_session will be different if a user takes a rest and come back after a while, which means 1 order session/user_id may have different user_sessions. 

Event_type including view, cart, remove from cart and purchase. Each product presents one event, so if one order has multiple products, it will have multiple purchase events at same time

Category code and brand are not fully recorded. Present for meaningful categories and skipped for different kinds of accessories. So the missing parts won't be dropped out.

In [4]:
df=pd.read_csv('2019-Dec.csv')
df

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-12-01 00:00:00 UTC,remove_from_cart,5712790,1487580005268456287,NaN,f.o.x,6.27,576802932,51d85cb0-897f-48d2-918b-ad63965c12dc
1,2019-12-01 00:00:00 UTC,view,5764655,1487580005411062629,NaN,cnd,29.05,412120092,8adff31e-2051-4894-9758-224bfa8aec18
2,2019-12-01 00:00:02 UTC,cart,4958,1487580009471148064,NaN,runail,1.19,494077766,c99a50e8-2fac-4c4d-89ec-41c05f114554
3,2019-12-01 00:00:05 UTC,view,5848413,1487580007675986893,NaN,freedecor,0.79,348405118,722ffea5-73c0-4924-8e8f-371ff8031af4
4,2019-12-01 00:00:07 UTC,view,5824148,1487580005511725929,NaN,NaN,5.56,576005683,28172809-7e4a-45ce-bab0-5efa90117cd5
...,...,...,...,...,...,...,...,...,...
3533281,2019-12-31 23:59:39 UTC,view,5683350,1487580005671109489,NaN,masura,2.84,536812729,e4a2d47c-a956-4c46-8176-745f52ea664b
3533282,2019-12-31 23:59:46 UTC,view,5888097,1487580013388628160,NaN,shik,179.05,503658154,2dde9867-9e71-4a64-880d-aa68b66aae6d
3533283,2019-12-31 23:59:51 UTC,view,59975,1487580012096782476,NaN,NaN,7.14,595414541,4c6d80bb-5dd3-4fbb-b592-187b51db2753
3533284,2019-12-31 23:59:52 UTC,view,5775982,1783999063314661546,NaN,NaN,11.90,397780878,7e8a2b85-153a-44eb-a71f-b748fde14fcc


Check all columns and drop those rows with neg prices

In [5]:
df['category_code'].value_counts(dropna=False)

NaN                                       3474821
appliances.environment.vacuum               25818
stationery.cartrige                          9500
apparel.glove                                8295
accessories.bag                              4938
furniture.living_room.cabinet                4478
furniture.bathroom.bath                      3529
appliances.personal.hair_cutter              1102
accessories.cosmetic_bag                      693
appliances.environment.air_conditioner         92
furniture.living_room.chair                    20
Name: category_code, dtype: int64

In [6]:
df['event_type'].value_counts(dropna=False)

view                1728331
cart                 927124
remove_from_cart     664655
purchase             213176
Name: event_type, dtype: int64

In [7]:
df['user_id'].value_counts(dropna=False)

527021202    8689
572631412    3883
467810091    3147
548810441    2426
584289031    2356
             ... 
586739138       1
590937536       1
594889500       1
578079532       1
529467934       1
Name: user_id, Length: 370154, dtype: int64

In [8]:
df['brand'].value_counts(dropna=False)

NaN          1510289
runail        256217
grattol       155920
irisk         152518
masura        125185
              ...   
ikoo               3
macadamia          1
ibd                1
shifei             1
dessata            1
Name: brand, Length: 253, dtype: int64

In [9]:
df['price'].value_counts(dropna=False,bins=10)

(2.06, 42.775]        2517173
(-38.655, 2.06]        873690
(42.775, 83.49]         96951
(83.49, 124.205]        22346
(124.205, 164.92]        9873
(164.92, 205.635]        8351
(287.065, 327.78]        1972
(205.635, 246.35]        1727
(246.35, 287.065]        1197
(-79.778, -38.655]          6
Name: price, dtype: int64

In [10]:
df['user_session'].value_counts(dropna=False)

d53d1c2e-9195-6051-cf7b-7332dfeae818    1393
4f70385c-0169-476f-aa1e-5dc8937817ab    1336
08868af8-a0ec-4fcf-ae7f-1aab30d0084b    1270
1ea0524e-3745-430b-8fca-fbd3f552e816    1050
a062f07b-4eef-4266-8660-b3040ed412aa     940
                                        ... 
0b739922-ff43-4b8c-b5cb-6d389ff8b49a       1
c349c990-1e40-4984-b544-b3d850e4c6ee       1
dcabc247-79a5-448f-aab4-e3e071e8653f       1
93392e8e-b5e2-4c1a-ab96-a2758051b0bf       1
226bc916-dd42-4016-8ef7-540738a57f86       1
Name: user_session, Length: 839813, dtype: int64

In [11]:
df[(df['price']<0)]

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
693061,2019-12-06 08:44:06 UTC,purchase,5670257,1487580014042939619,NaN,NaN,-15.87,410016187,2414f145-f7fa-473f-9d74-41e5694178a0
942049,2019-12-08 12:38:35 UTC,purchase,5670257,1487580014042939619,NaN,NaN,-15.87,579811230,001d3051-07af-1cef-f5e4-10443d9742b9
1633142,2019-12-13 08:44:49 UTC,purchase,5716859,1487580014042939619,NaN,NaN,-47.62,414087247,6617078d-899a-4b89-8538-b70e505bff38
1667368,2019-12-13 13:48:10 UTC,purchase,5716857,1487580014042939619,NaN,NaN,-23.81,584773479,a3f156d9-b7fc-444d-8899-920715cca75d
2143701,2019-12-17 10:37:03 UTC,purchase,5670257,1487580014042939619,NaN,NaN,-15.87,584498077,ecbf0f7c-3c10-42e8-9af6-1397b5428afa
2143702,2019-12-17 10:37:03 UTC,purchase,5716855,1487580014042939619,NaN,NaN,-7.94,584498077,ecbf0f7c-3c10-42e8-9af6-1397b5428afa
2164651,2019-12-17 13:31:01 UTC,purchase,5670257,1487580014042939619,NaN,NaN,-15.87,513455730,efd60b1a-f616-4f48-a5d2-1a1ab0bd61e1
2548146,2019-12-20 17:56:16 UTC,purchase,5716859,1487580014042939619,NaN,NaN,-47.62,589538798,094f5f94-e7d2-4d05-a722-dbc2015c0351
2754627,2019-12-22 19:31:01 UTC,purchase,5716859,1487580014042939619,NaN,NaN,-47.62,590958276,42953b86-32c5-4861-8760-e9c2cd35a609
3053899,2019-12-25 13:01:21 UTC,purchase,5716861,1487580014042939619,NaN,NaN,-79.37,489184067,af1b35db-1024-495e-8c42-b959491ba136


In [12]:
df=df.drop(df[df.price<0].index)

In [13]:
df

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-12-01 00:00:00 UTC,remove_from_cart,5712790,1487580005268456287,NaN,f.o.x,6.27,576802932,51d85cb0-897f-48d2-918b-ad63965c12dc
1,2019-12-01 00:00:00 UTC,view,5764655,1487580005411062629,NaN,cnd,29.05,412120092,8adff31e-2051-4894-9758-224bfa8aec18
2,2019-12-01 00:00:02 UTC,cart,4958,1487580009471148064,NaN,runail,1.19,494077766,c99a50e8-2fac-4c4d-89ec-41c05f114554
3,2019-12-01 00:00:05 UTC,view,5848413,1487580007675986893,NaN,freedecor,0.79,348405118,722ffea5-73c0-4924-8e8f-371ff8031af4
4,2019-12-01 00:00:07 UTC,view,5824148,1487580005511725929,NaN,NaN,5.56,576005683,28172809-7e4a-45ce-bab0-5efa90117cd5
...,...,...,...,...,...,...,...,...,...
3533281,2019-12-31 23:59:39 UTC,view,5683350,1487580005671109489,NaN,masura,2.84,536812729,e4a2d47c-a956-4c46-8176-745f52ea664b
3533282,2019-12-31 23:59:46 UTC,view,5888097,1487580013388628160,NaN,shik,179.05,503658154,2dde9867-9e71-4a64-880d-aa68b66aae6d
3533283,2019-12-31 23:59:51 UTC,view,59975,1487580012096782476,NaN,NaN,7.14,595414541,4c6d80bb-5dd3-4fbb-b592-187b51db2753
3533284,2019-12-31 23:59:52 UTC,view,5775982,1783999063314661546,NaN,NaN,11.90,397780878,7e8a2b85-153a-44eb-a71f-b748fde14fcc


Take a smaple with 50k rows. First clean the event_time with right time frame, then groupby sessions with purchase event only and canculate time difference, so that we can know how long one purchase session takes.

In [14]:
sample=df.loc[:50000,:]
sample

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-12-01 00:00:00 UTC,remove_from_cart,5712790,1487580005268456287,NaN,f.o.x,6.27,576802932,51d85cb0-897f-48d2-918b-ad63965c12dc
1,2019-12-01 00:00:00 UTC,view,5764655,1487580005411062629,NaN,cnd,29.05,412120092,8adff31e-2051-4894-9758-224bfa8aec18
2,2019-12-01 00:00:02 UTC,cart,4958,1487580009471148064,NaN,runail,1.19,494077766,c99a50e8-2fac-4c4d-89ec-41c05f114554
3,2019-12-01 00:00:05 UTC,view,5848413,1487580007675986893,NaN,freedecor,0.79,348405118,722ffea5-73c0-4924-8e8f-371ff8031af4
4,2019-12-01 00:00:07 UTC,view,5824148,1487580005511725929,NaN,NaN,5.56,576005683,28172809-7e4a-45ce-bab0-5efa90117cd5
...,...,...,...,...,...,...,...,...,...
49996,2019-12-01 12:26:05 UTC,remove_from_cart,5820728,1487580006317032337,NaN,NaN,2.22,580187026,d1ac0aa1-0604-4022-89c5-213e75215f4a
49997,2019-12-01 12:26:05 UTC,view,5802651,1487580005754995573,NaN,NaN,3.89,573545424,9402c32f-c637-4405-a270-132cd95483d9
49998,2019-12-01 12:26:05 UTC,view,5694897,1487580009076883471,NaN,NaN,1.27,580244225,cc6baeb2-601c-4e0e-9536-e6c1fc818e97
49999,2019-12-01 12:26:06 UTC,cart,5814841,1487580005134238553,NaN,runail,2.38,544849588,a934334c-03fb-478a-b678-579fdaed07ab


Convert time to other information: month, day, daytime

In [15]:
pd.to_datetime(sample['event_time'],format='%Y-%m-%d %H:%M:%S',errors='ignore')

0        2019-12-01 00:00:00 UTC
1        2019-12-01 00:00:00 UTC
2        2019-12-01 00:00:02 UTC
3        2019-12-01 00:00:05 UTC
4        2019-12-01 00:00:07 UTC
                  ...           
49996    2019-12-01 12:26:05 UTC
49997    2019-12-01 12:26:05 UTC
49998    2019-12-01 12:26:05 UTC
49999    2019-12-01 12:26:06 UTC
50000    2019-12-01 12:26:06 UTC
Name: event_time, Length: 50001, dtype: object

In [16]:
sample['time']=pd.DataFrame(sample['event_time'].str.split('UTC').str[0])

In [17]:
sample['time']=pd.to_datetime(sample['time'],format='%Y-%m-%d %H:%M:%S',errors='ignore')

In [18]:
sample['month']=sample['time'].dt.month
sample['day']=sample['time'].dt.day
sample['hour']=sample['time'].dt.hour

In [19]:
sample

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,time,month,day,hour
0,2019-12-01 00:00:00 UTC,remove_from_cart,5712790,1487580005268456287,NaN,f.o.x,6.27,576802932,51d85cb0-897f-48d2-918b-ad63965c12dc,2019-12-01 00:00:00,12,1,0
1,2019-12-01 00:00:00 UTC,view,5764655,1487580005411062629,NaN,cnd,29.05,412120092,8adff31e-2051-4894-9758-224bfa8aec18,2019-12-01 00:00:00,12,1,0
2,2019-12-01 00:00:02 UTC,cart,4958,1487580009471148064,NaN,runail,1.19,494077766,c99a50e8-2fac-4c4d-89ec-41c05f114554,2019-12-01 00:00:02,12,1,0
3,2019-12-01 00:00:05 UTC,view,5848413,1487580007675986893,NaN,freedecor,0.79,348405118,722ffea5-73c0-4924-8e8f-371ff8031af4,2019-12-01 00:00:05,12,1,0
4,2019-12-01 00:00:07 UTC,view,5824148,1487580005511725929,NaN,NaN,5.56,576005683,28172809-7e4a-45ce-bab0-5efa90117cd5,2019-12-01 00:00:07,12,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49996,2019-12-01 12:26:05 UTC,remove_from_cart,5820728,1487580006317032337,NaN,NaN,2.22,580187026,d1ac0aa1-0604-4022-89c5-213e75215f4a,2019-12-01 12:26:05,12,1,12
49997,2019-12-01 12:26:05 UTC,view,5802651,1487580005754995573,NaN,NaN,3.89,573545424,9402c32f-c637-4405-a270-132cd95483d9,2019-12-01 12:26:05,12,1,12
49998,2019-12-01 12:26:05 UTC,view,5694897,1487580009076883471,NaN,NaN,1.27,580244225,cc6baeb2-601c-4e0e-9536-e6c1fc818e97,2019-12-01 12:26:05,12,1,12
49999,2019-12-01 12:26:06 UTC,cart,5814841,1487580005134238553,NaN,runail,2.38,544849588,a934334c-03fb-478a-b678-579fdaed07ab,2019-12-01 12:26:06,12,1,12


In [20]:
sample=sample.drop(columns='event_time')

Because in price column, it recodes price for all different events. To select prices from the products only been purchased, use the last_event=purchase as condition. Same for number of products in one order

In [21]:
sample['max_time']=sample['time']
sample['last_event']=sample['event_type']

In [22]:
sample['amount']=np.where(sample['last_event']=='purchase',sample['price'],0)

In [23]:
sample['nr_product']=np.where(sample['last_event']=='purchase',1,0)
sample

,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,time,month,day,hour,max_time,last_event,amount,nr_product
0,remove_from_cart,5712790,1487580005268456287,NaN,f.o.x,6.27,576802932,51d85cb0-897f-48d2-918b-ad63965c12dc,2019-12-01 00:00:00,12,1,0,2019-12-01 00:00:00,remove_from_cart,0.0,0
1,view,5764655,1487580005411062629,NaN,cnd,29.05,412120092,8adff31e-2051-4894-9758-224bfa8aec18,2019-12-01 00:00:00,12,1,0,2019-12-01 00:00:00,view,0.0,0
2,cart,4958,1487580009471148064,NaN,runail,1.19,494077766,c99a50e8-2fac-4c4d-89ec-41c05f114554,2019-12-01 00:00:02,12,1,0,2019-12-01 00:00:02,cart,0.0,0
3,view,5848413,1487580007675986893,NaN,freedecor,0.79,348405118,722ffea5-73c0-4924-8e8f-371ff8031af4,2019-12-01 00:00:05,12,1,0,2019-12-01 00:00:05,view,0.0,0
4,view,5824148,1487580005511725929,NaN,NaN,5.56,576005683,28172809-7e4a-45ce-bab0-5efa90117cd5,2019-12-01 00:00:07,12,1,0,2019-12-01 00:00:07,view,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49996,remove_from_cart,5820728,1487580006317032337,NaN,NaN,2.22,580187026,d1ac0aa1-0604-4022-89c5-213e75215f4a,2019-12-01 12:26:05,12,1,12,2019-12-01 12:26:05,remove_from_cart,0.0,0
49997,view,5802651,1487580005754995573,NaN,NaN,3.89,573545424,9402c32f-c637-4405-a270-132cd95483d9,2019-12-01 12:26:05,12,1,12,2019-12-01 12:26:05,view,0.0,0
49998,view,5694897,1487580009076883471,NaN,NaN,1.27,580244225,cc6baeb2-601c-4e0e-9536-e6c1fc818e97,2019-12-01 12:26:05,12,1,12,2019-12-01 12:26:05,view,0.0,0
49999,cart,5814841,1487580005134238553,NaN,runail,2.38,544849588,a934334c-03fb-478a-b678-579fdaed07ab,2019-12-01 12:26:06,12,1,12,2019-12-01 12:26:06,cart,0.0,0


Group user id and session id to tell how many sessions been made by each user, with the starting time and finishing time.

In [24]:
sample_group_user=sample.groupby((['user_id','user_session']),as_index=False).agg({'time':'min',
                                                                                   'max_time':'max','amount':'sum',
                                                                                   'nr_product':'sum','event_type':'first',
                                                                                   'month':'first','day':'first',
                                                                                   'hour':'first','last_event':'last'})
sample_group_user['rank']=sample_group_user['time'].groupby(sample_group_user['user_id']).rank(ascending=1,method='dense')

In [25]:
sample_group_user

,user_id,user_session,time,max_time,amount,nr_product,event_type,month,day,hour,last_event,rank
0,31926511,3e97a9d0-2ef7-41e7-951c-f8522b4973be,2019-12-01 10:41:16,2019-12-01 10:41:23,0.0,0,remove_from_cart,12,1,10,view,1.0
1,31926511,a0ba0697-cb60-45b7-ae91-6b6784b30d32,2019-12-01 10:44:46,2019-12-01 10:50:35,0.0,0,view,12,1,10,view,2.0
2,48182054,07ed37dd-be3c-492b-bf31-8b7c3979bd4e,2019-12-01 10:14:20,2019-12-01 10:14:20,0.0,0,view,12,1,10,view,3.0
3,48182054,4bc81430-a486-466f-8bc6-738b1abf79f8,2019-12-01 09:39:41,2019-12-01 10:13:16,0.0,0,view,12,1,9,view,1.0
4,48182054,a8f715d0-a343-4564-a0f3-5531d88b58f7,2019-12-01 09:47:52,2019-12-01 09:47:52,0.0,0,view,12,1,9,view,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
12345,580243638,d6fd2440-9af4-42af-a8c0-3b0427acc3b7,2019-12-01 12:25:40,2019-12-01 12:25:40,0.0,0,view,12,1,12,view,1.0
12346,580243874,4ce5cbb7-ebd2-4f63-b8f4-41fa2ea2c0f9,2019-12-01 12:25:19,2019-12-01 12:25:19,0.0,0,view,12,1,12,view,1.0
12347,580243907,a2e3efc3-7a29-43b9-9616-645b0142fa9d,2019-12-01 12:25:23,2019-12-01 12:25:23,0.0,0,view,12,1,12,view,1.0
12348,580244189,7c0b1106-ca7b-456d-9892-c26fce4b079d,2019-12-01 12:26:00,2019-12-01 12:26:00,0.0,0,view,12,1,12,view,1.0


In [26]:
sample_group_user['time_diff']=(pd.to_datetime(sample_group_user['max_time'])-pd.to_datetime(sample_group_user['time'])).apply(lambda x: x.total_seconds())
sample_group_user

,user_id,user_session,time,max_time,amount,nr_product,event_type,month,day,hour,last_event,rank,time_diff
0,31926511,3e97a9d0-2ef7-41e7-951c-f8522b4973be,2019-12-01 10:41:16,2019-12-01 10:41:23,0.0,0,remove_from_cart,12,1,10,view,1.0,7.0
1,31926511,a0ba0697-cb60-45b7-ae91-6b6784b30d32,2019-12-01 10:44:46,2019-12-01 10:50:35,0.0,0,view,12,1,10,view,2.0,349.0
2,48182054,07ed37dd-be3c-492b-bf31-8b7c3979bd4e,2019-12-01 10:14:20,2019-12-01 10:14:20,0.0,0,view,12,1,10,view,3.0,0.0
3,48182054,4bc81430-a486-466f-8bc6-738b1abf79f8,2019-12-01 09:39:41,2019-12-01 10:13:16,0.0,0,view,12,1,9,view,1.0,2015.0
4,48182054,a8f715d0-a343-4564-a0f3-5531d88b58f7,2019-12-01 09:47:52,2019-12-01 09:47:52,0.0,0,view,12,1,9,view,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12345,580243638,d6fd2440-9af4-42af-a8c0-3b0427acc3b7,2019-12-01 12:25:40,2019-12-01 12:25:40,0.0,0,view,12,1,12,view,1.0,0.0
12346,580243874,4ce5cbb7-ebd2-4f63-b8f4-41fa2ea2c0f9,2019-12-01 12:25:19,2019-12-01 12:25:19,0.0,0,view,12,1,12,view,1.0,0.0
12347,580243907,a2e3efc3-7a29-43b9-9616-645b0142fa9d,2019-12-01 12:25:23,2019-12-01 12:25:23,0.0,0,view,12,1,12,view,1.0,0.0
12348,580244189,7c0b1106-ca7b-456d-9892-c26fce4b079d,2019-12-01 12:26:00,2019-12-01 12:26:00,0.0,0,view,12,1,12,view,1.0,0.0


In [27]:
sample_group_user[sample_group_user['user_id']==473613801]

,user_id,user_session,time,max_time,amount,nr_product,event_type,month,day,hour,last_event,rank,time_diff
1890,473613801,d1e90c6a-c030-4d06-aaaf-6d12ec502bb9,2019-12-01 00:09:55,2019-12-01 00:17:50,19.18,25,cart,12,1,0,purchase,1.0,475.0


In [28]:
order_session_dict=sample_group_user['user_id'].value_counts().to_dict()
order_session_dict

{527021202: 662,
 476577170: 306,
 576802932: 289,
 579733537: 88,
 487506891: 83,
 579179073: 74,
 577896990: 71,
 557616099: 66,
 534143009: 63,
 231412385: 54,
 576648112: 46,
 570477322: 43,
 534779173: 38,
 580143586: 37,
 574032814: 37,
 551195949: 35,
 552408840: 33,
 576872422: 33,
 470598515: 31,
 556730134: 24,
 574478482: 23,
 423200429: 23,
 509211181: 23,
 269904158: 22,
 580167541: 22,
 580056441: 21,
 571987866: 21,
 567890777: 20,
 503730648: 19,
 579183835: 18,
 577872267: 18,
 574974492: 17,
 504266205: 17,
 579982297: 17,
 455300469: 17,
 306976352: 16,
 560735538: 16,
 558093005: 15,
 580154910: 15,
 509647314: 14,
 574710774: 14,
 579928580: 14,
 568841081: 14,
 331850715: 13,
 572268713: 13,
 461862371: 13,
 559553156: 13,
 532955590: 12,
 417812786: 12,
 578857321: 12,
 509118747: 12,
 580222011: 12,
 570823513: 12,
 251717161: 12,
 557713455: 11,
 575646009: 11,
 580049124: 11,
 399967334: 11,
 522537939: 10,
 547167413: 10,
 568407203: 10,
 532763617: 10,
 5456

There are several different cases: a user could finish one order without interrupt, or left computer and come back only click 'purchase'(then it would be 2 session with the 2nd session only has purchase event), or come back click something and then purchase. Thus need to add sesstion times from different sessions if they are not ending with 'purchase'. If they have multiple sessions with multiple 'purchase', then seperate them as multiple orders

In [29]:
user_id=[]
order_time=[]
event=[]
total_price=[]
number=[]
log_month=[]
log_day=[]
log_hour=[]

for user,num in tqdm(order_session_dict.items()):
    if num ==1:
        user_id.append(user)
        order_time.append(list(sample_group_user[sample_group_user['user_id']==user]['time_diff'])[0])
        event.append(list(sample_group_user[sample_group_user['user_id']==user]['last_event'])[0])
        total_price.append(list(sample_group_user[sample_group_user['user_id']==user]['amount'])[0])
        number.append(list(sample_group_user[sample_group_user['user_id']==user]['rank'])[0])
        log_month.append(list(sample_group_user[sample_group_user['user_id']==user]['month'])[0])
        log_day.append(list(sample_group_user[sample_group_user['user_id']==user]['day'])[0])
        log_hour.append(list(sample_group_user[sample_group_user['user_id']==user]['hour'])[0])
    else:
        totle_time=0
        for i in range(num):
            session_time=list(sample_group_user[sample_group_user['user_id']==user]['time_diff'])[i]
            totle_time+=session_time
            user_id.append(user)
            order_time.append(totle_time)
            event.append(list(sample_group_user[sample_group_user['user_id']==user]['last_event'])[i])
            total_price.append(list(sample_group_user[sample_group_user['user_id']==user]['amount'])[i])
            number.append(list(sample_group_user[sample_group_user['user_id']==user]['rank'])[i])
            log_month.append(list(sample_group_user[sample_group_user['user_id']==user]['month'])[i])
            log_day.append(list(sample_group_user[sample_group_user['user_id']==user]['day'])[i])
            log_hour.append(list(sample_group_user[sample_group_user['user_id']==user]['hour'])[i])

In [31]:
new_df=pd.DataFrame({'user_id':user_id,'spent_time':order_time,'month':log_month,'day':log_day,'hour':log_hour,'last_event':event,'purchase_amount':total_price,'rank':number})
new_df

,user_id,spent_time,month,day,hour,last_event,purchase_amount,rank
0,527021202,0.0,12,1,11,view,0.0,205.0
1,527021202,106.0,12,1,12,view,0.0,518.0
2,527021202,106.0,12,1,12,view,0.0,470.0
3,527021202,106.0,12,1,12,view,0.0,562.0
4,527021202,106.0,12,1,11,view,0.0,318.0
...,...,...,...,...,...,...,...,...
12345,545056451,0.0,12,1,7,view,0.0,1.0
12346,531458184,0.0,12,1,10,view,0.0,1.0
12347,449362058,0.0,12,1,11,view,0.0,1.0
12348,380694667,0.0,12,1,11,view,0.0,1.0


In [32]:
new_df['last_event'].value_counts()

view                10429
cart                 1149
remove_from_cart      502
purchase              270
Name: last_event, dtype: int64

In [33]:
new_df.sort_values(by=['user_id','rank'],inplace=False)

,user_id,spent_time,month,day,hour,last_event,purchase_amount,rank
5286,31926511,7.0,12,1,10,view,0.0,1.0
5287,31926511,356.0,12,1,10,view,0.0,2.0
4065,48182054,2015.0,12,1,9,view,0.0,1.0
4066,48182054,2015.0,12,1,9,view,0.0,2.0
4064,48182054,0.0,12,1,10,view,0.0,3.0
...,...,...,...,...,...,...,...,...
11415,580243638,0.0,12,1,12,view,0.0,1.0
7987,580243874,0.0,12,1,12,view,0.0,1.0
8733,580243907,0.0,12,1,12,view,0.0,1.0
8679,580244189,0.0,12,1,12,view,0.0,1.0


In [34]:
new_df['sessiontime']=0

In [35]:
time = 0
for i in range(len(new_df)):
    if new_df['last_event'].iloc[i] != 'purchase':
        time += new_df['spent_time'].iloc[i]
    else:
        time += new_df['spent_time'].iloc[i]
        new_df['sessiontime'].iloc[i] = time
        time = 0
        

In [36]:
new_df[new_df['sessiontime']!=0]

,user_id,spent_time,month,day,hour,last_event,purchase_amount,rank,sessiontime
1184,576802932,5404.0,12,1,0,purchase,233.41,242.0,7888223.0
1660,534143009,4498.0,12,1,10,purchase,14.61,34.0,808958.0
1856,534779173,2644.0,12,1,8,purchase,54.19,37.0,629302.0
2062,470598515,2901.0,12,1,9,purchase,133.01,27.0,1857450.0
2177,509211181,9419.0,12,1,11,purchase,66.45,21.0,285911.0
...,...,...,...,...,...,...,...,...,...
12056,368287207,12448.0,12,1,7,purchase,22.21,1.0,33496.0
12084,572450208,0.0,12,1,8,purchase,21.27,1.0,2667.0
12105,579373654,1016.0,12,1,3,purchase,16.98,1.0,6911.0
12150,454947115,181.0,12,1,10,purchase,5.96,1.0,6180.0


Wrote functions and put all data into it

In [30]:
def clean_columns(df):
    df['time']=pd.DataFrame(df['event_time'].str.split('UTC').str[0])
    df['time']=pd.to_datetime(df['time'],format='%Y-%m-%d %H:%M:%S',errors='ignore')

    df['month']=df['time'].dt.month
    df['day']=df['time'].dt.day
    df['hour']=df['time'].dt.hour

    df=df.drop(columns='event_time')

    df['max_time']=df['time']
    df['last_event']=df['event_type']
    df['amount']=np.where(df['last_event']=='purchase',df['price'],0)
    df['nr_product']=np.where(df['last_event']=='purchase',1,0)
    return df

In [31]:
df=clean_columns(df)
df

,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,time,month,day,hour,max_time,last_event,amount,nr_product
0,remove_from_cart,5712790,1487580005268456287,NaN,f.o.x,6.27,576802932,51d85cb0-897f-48d2-918b-ad63965c12dc,2019-12-01 00:00:00,12,1,0,2019-12-01 00:00:00,remove_from_cart,0.0,0
1,view,5764655,1487580005411062629,NaN,cnd,29.05,412120092,8adff31e-2051-4894-9758-224bfa8aec18,2019-12-01 00:00:00,12,1,0,2019-12-01 00:00:00,view,0.0,0
2,cart,4958,1487580009471148064,NaN,runail,1.19,494077766,c99a50e8-2fac-4c4d-89ec-41c05f114554,2019-12-01 00:00:02,12,1,0,2019-12-01 00:00:02,cart,0.0,0
3,view,5848413,1487580007675986893,NaN,freedecor,0.79,348405118,722ffea5-73c0-4924-8e8f-371ff8031af4,2019-12-01 00:00:05,12,1,0,2019-12-01 00:00:05,view,0.0,0
4,view,5824148,1487580005511725929,NaN,NaN,5.56,576005683,28172809-7e4a-45ce-bab0-5efa90117cd5,2019-12-01 00:00:07,12,1,0,2019-12-01 00:00:07,view,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3533281,view,5683350,1487580005671109489,NaN,masura,2.84,536812729,e4a2d47c-a956-4c46-8176-745f52ea664b,2019-12-31 23:59:39,12,31,23,2019-12-31 23:59:39,view,0.0,0
3533282,view,5888097,1487580013388628160,NaN,shik,179.05,503658154,2dde9867-9e71-4a64-880d-aa68b66aae6d,2019-12-31 23:59:46,12,31,23,2019-12-31 23:59:46,view,0.0,0
3533283,view,59975,1487580012096782476,NaN,NaN,7.14,595414541,4c6d80bb-5dd3-4fbb-b592-187b51db2753,2019-12-31 23:59:51,12,31,23,2019-12-31 23:59:51,view,0.0,0
3533284,view,5775982,1783999063314661546,NaN,NaN,11.90,397780878,7e8a2b85-153a-44eb-a71f-b748fde14fcc,2019-12-31 23:59:52,12,31,23,2019-12-31 23:59:52,view,0.0,0


In [32]:
def group_by(df):
    df_user=df.groupby((['user_id','user_session']),as_index=False).agg({'time':'min','max_time':'max','amount':'sum',
                                                                        'nr_product':'sum','event_type':'first',
                                                                         'month':'first','day':'first',
                                                                        'hour':'first','last_event':'last'})
    df_user['rank']=df_user['time'].groupby(df_user['user_id']).rank(ascending=1,method='dense')
    df_user['time_diff']=(pd.to_datetime(df_user['max_time'])-pd.to_datetime(df_user['time'])).apply(lambda x: x.total_seconds())
    return df_user

In [33]:
df=group_by(df)
df

,user_id,user_session,time,max_time,amount,nr_product,event_type,month,day,hour,last_event,rank,time_diff
0,1180452,a4818e6d-9069-4aa8-8731-572ac266283f,2019-12-28 14:32:56,2019-12-28 14:32:56,0.0,0,view,12,28,14,view,1.0,0.0
1,2963072,3bf2bbbb-1a32-4f06-a263-a3f49db74750,2019-12-22 12:50:22,2019-12-22 13:36:13,0.0,0,view,12,22,12,view,1.0,2751.0
2,4661182,758d0b2c-abc8-406f-b87b-b25ba05f670e,2019-12-22 19:32:12,2019-12-22 19:32:49,0.0,0,view,12,22,19,view,2.0,37.0
3,4661182,78b9e338-30d6-467c-820e-529e5dd1e184,2019-12-06 10:22:41,2019-12-06 10:22:41,0.0,0,cart,12,6,10,cart,1.0,0.0
4,4891613,538cda82-30a9-4408-9596-1098bbb195a4,2019-12-25 22:16:25,2019-12-25 22:16:25,0.0,0,view,12,25,22,view,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
839860,595413843,468fcf5a-56d4-4ec2-a73e-5403256fdd22,2019-12-31 23:55:52,2019-12-31 23:55:52,0.0,0,view,12,31,23,view,1.0,0.0
839861,595413976,e29832b3-239a-491f-8fad-220fd3f1a5e8,2019-12-31 23:55:33,2019-12-31 23:55:33,0.0,0,view,12,31,23,view,1.0,0.0
839862,595414210,ebd2969c-dce1-451a-af8a-dce36c2e1687,2019-12-31 23:57:08,2019-12-31 23:57:08,0.0,0,view,12,31,23,view,1.0,0.0
839863,595414257,2d8ea6a9-102c-49be-85e3-dc631b584606,2019-12-31 23:57:48,2019-12-31 23:57:48,0.0,0,view,12,31,23,view,1.0,0.0


In [34]:
def user_list(df):
        order_session_dict=df['user_id'].value_counts().to_dict()

        user_id=[]
        order_time=[]
        event=[]
        total_price=[]
        product=[]
        number=[]
        log_month=[]
        log_day=[]
        log_hour=[]

        for user,num in tqdm(order_session_dict.items()):
            if num ==1:
                user_id.append(user)
                order_time.append(list(df[df['user_id']==user]['time_diff'])[0])
                event.append(list(df[df['user_id']==user]['last_event'])[0])
                total_price.append(list(df[df['user_id']==user]['amount'])[0])
                product.append(list(df[df['user_id']==user]['nr_product'])[0])
                number.append(list(df[df['user_id']==user]['rank'])[0])
                log_month.append(list(df[df['user_id']==user]['month'])[0])
                log_day.append(list(df[df['user_id']==user]['day'])[0])
                log_hour.append(list(df[df['user_id']==user]['hour'])[0])
            
            
            else:
                totle_time=0
                for i in range(num):
                    session_time=list(df[df['user_id']==user]['time_diff'])[i]
                    totle_time+=session_time
                    user_id.append(user)
                    order_time.append(totle_time)
                    event.append(list(df[df['user_id']==user]['last_event'])[i])
                    total_price.append(list(df[df['user_id']==user]['amount'])[i])
                    product.append(list(df[df['user_id']==user]['nr_product'])[i])
                    number.append(list(df[df['user_id']==user]['rank'])[i])
                    log_month.append(list(df[df['user_id']==user]['month'])[i])
                    log_day.append(list(df[df['user_id']==user]['day'])[i])
                    log_hour.append(list(df[df['user_id']==user]['hour'])[i])
        
        new_df=pd.DataFrame({'user_id':user_id,'spent_time':order_time,'month':log_month,'day':log_day,'hour':log_hour,'last_event':event,'purchase_amount':total_price,'nr_product':product,'rank':number})
        
        return new_df.sort_values(by=['user_id','rank'],inplace=False)

In [35]:
user_session=user_list(df)
user_session

,user_id,spent_time,month,day,hour,last_event,purchase_amount,nr_product,rank
816439,1180452,0.0,12,28,14,view,0.0,0,1.0
823013,2963072,2751.0,12,22,12,view,0.0,0,1.0
471874,4661182,37.0,12,6,10,cart,0.0,0,1.0
471873,4661182,37.0,12,22,19,view,0.0,0,2.0
671232,4891613,0.0,12,25,22,view,0.0,0,1.0
...,...,...,...,...,...,...,...,...,...
652038,595413843,0.0,12,31,23,view,0.0,0,1.0
688165,595413976,0.0,12,31,23,view,0.0,0,1.0
714531,595414210,0.0,12,31,23,view,0.0,0,1.0
610129,595414257,0.0,12,31,23,view,0.0,0,1.0


In [ ]:
user_session[user_session['user_id']==473613801]

In [36]:
user_session.sort_values(by=['day','user_id'],inplace=True)

In [37]:
def event_purchase(df):
    df['sessiontime']=0
    time = 0
    
    for i in tqdm(range(len(df)-1)):
        if df['user_id'].iloc[i]== df['user_id'].iloc[i+1]:
            time += df['spent_time'].iloc[i]
        else:
            time += df['spent_time'].iloc[i]
            df['sessiontime'].iloc[i] = time
            time = 0
    
    return df[df['sessiontime']!=0]

In [38]:
sessions=event_purchase(user_session)
sessions

,user_id,spent_time,month,day,hour,last_event,purchase_amount,nr_product,rank,sessiontime
379503,9794320,308.0,12,1,19,view,0.00,0,1.0,308.0
500084,31926511,356.0,12,1,10,view,0.00,0,2.0,363.0
643045,39007293,95.0,12,1,19,view,0.00,0,1.0,95.0
614510,47076222,154304.0,12,1,22,view,0.00,0,1.0,154304.0
419814,48182054,0.0,12,1,10,view,0.00,0,3.0,4030.0
...,...,...,...,...,...,...,...,...,...,...
618580,595411525,108.0,12,31,23,view,0.00,0,1.0,108.0
374356,595411904,125.0,12,31,23,view,0.00,0,4.0,465.0
713300,595412191,251.0,12,31,23,cart,0.00,0,1.0,251.0
804770,595412617,706.0,12,31,23,view,0.00,0,1.0,706.0


In [39]:
sessions.sort_values(by=['day','hour'],inplace=True)

In [40]:
sessions

,user_id,spent_time,month,day,hour,last_event,purchase_amount,nr_product,rank,sessiontime
355984,234044775,2887.0,12,1,0,purchase,41.39,5,1.0,2887.0
341079,252159723,1806.0,12,1,0,view,0.00,0,2.0,3360.0
590716,259408911,154.0,12,1,0,view,0.00,0,1.0,154.0
566609,348405118,464.0,12,1,0,cart,0.00,0,1.0,464.0
424132,356011474,2923.0,12,1,0,purchase,56.91,31,1.0,2923.0
...,...,...,...,...,...,...,...,...,...,...
618580,595411525,108.0,12,31,23,view,0.00,0,1.0,108.0
374356,595411904,125.0,12,31,23,view,0.00,0,4.0,465.0
713300,595412191,251.0,12,31,23,cart,0.00,0,1.0,251.0
804770,595412617,706.0,12,31,23,view,0.00,0,1.0,706.0


In [41]:
sessions.to_csv('sessions.csv')